# API
## Predict Interior Design Style
original source: https://towardsdatascience.com/colabcode-deploying-machine-learning-models-from-google-colab-54e0d37a7b09

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install colabcode

In [2]:
!pip install fastapi

     |████████████████████████████████| 52 kB 154 kB/s 
     |████████████████████████████████| 10.9 MB 9.9 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 


In [29]:
from fastapi import FastAPI
from PIL import Image
import tensorflow as tf
import pandas as pd
import requests

predict_id_style_app = FastAPI()

@predict_id_style_app.on_event("startup")
def load_model():
    global model
    model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/model')


@predict_id_style_app.get('/')
async def root():
  return {'message': 'API for predict interior design style based on photo image'}


@predict_id_style_app.get('/predict')
def predict_style(img_url: str):
  results = get_prediction(img_url)
  return {'prediction': results}


def get_prediction(_img_url):
  # variables
  _num_classes = 5
  _class_names = ['art deco', 'balinese', 'bohemian', 'industrial', 'minimalist']

  # download image
  _file_name = 'tmp_file'
  _r = requests.get(_img_url, allow_redirects=True)
  open(_file_name, 'w+b').write(_r.content)

  # load and resize image
  with Image.open(_file_name) as _img:
    _img = _img.resize((200, 200))

  # convert image to array
  _img_array = tf.keras.utils.img_to_array(_img)
  _img_array = tf.expand_dims(_img_array, 0)

  # perform prediction
  _predictions = model.predict(_img_array)
  _score = tf.nn.softmax(_predictions[0])

  # tabulate findings into dataframe
  _df = pd.DataFrame(_predictions.reshape(_num_classes,), index=_class_names, columns=['score'])
  _df['confidence'] = ['{:.2f}'.format(float(x) * 100) for x in _score]
  _df = _df.sort_values(by='score', ascending=False)

  # generate results
  _results = ''
  _entries = 0
  for _index, _confidence in enumerate(_df['confidence']):
    if float(_confidence) > 50:
      _entries += 1
      _results += _df.index[_index] + ': ' + _confidence + '%  '

  if _entries == 0:
    _results += _df.index[0] + ': ' + _df['confidence'][0] + '%  '
    _results += _df.index[1] + ': ' + _df['confidence'][1] + '%  '
  
  return _results



In [30]:
from colabcode import ColabCode
server = ColabCode(port=10000, code=False)

In [32]:
server.run_app(app=predict_id_style_app)

Public URL: NgrokTunnel: "https://fe7f-34-74-136-110.ngrok.io" -> "http://localhost:10000"


INFO:     Started server process [167]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:10000 (Press CTRL+C to quit)


INFO:     82.208.14.110:0 - "GET /predict?img_url=http%3A%2F%2Fcdn.home-designing.com%2Fwp-content%2Fuploads%2F2019%2F05%2Fmodern-sofa-1.jpg HTTP/1.1" 200 OK
INFO:     82.208.14.110:0 - "GET /predict?img_url=https%3A%2F%2Fwww.coasterfurniture.com%2Fwp-content%2Fuploads%2Fmodern-living-room.jpg HTTP/1.1" 200 OK
INFO:     82.208.14.111:0 - "GET /predict?img_url=https%3A%2F%2Fi.pinimg.com%2Foriginals%2Fd2%2F7e%2F97%2Fd27e9718701ffa36214559b7d08ab47c.jpg HTTP/1.1" 200 OK
INFO:     82.208.14.110:0 - "GET /predict?img_url=https%3A%2F%2Fwww.dmlights.com%2Fblog%2Fwp-content%2Fuploads%2F2015%2F12%2Fsobere-keuken.jpg HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [167]


### **End**